<a href="https://colab.research.google.com/github/uapdhyaybipul/Computer_vision/blob/main/bg_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install git+https://github.com/facebookresearch/segment-anything-2
!wget https://huggingface.co/facebook/sam2-hiera-large/resolve/main/sam2_hiera_large.pt -O /content/sam2_hiera_large.pt
!wget https://huggingface.co/facebook/sam2-hiera-large/raw/main/sam2_hiera_l.yaml -O /content/sam2_hiera_l.yaml
!wget https://files.catbox.moe/wt9v5h.mp4 -O /content/input.mp4


  Cloning https://github.com/facebookresearch/segment-anything-2 to /tmp/pip-req-build-lwpljq5q
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2 /tmp/pip-req-build-lwpljq5q
  Resolved https://github.com/facebookresearch/segment-anything-2 to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.0 MB/s eta 0:00:00
  Created wheel for SAM-2: filename=SAM_2-1.0-cp310-cp310-linux_x86_64.whl size=522720 sha256=98e3e2ec05746c0d4

--2024-12-23 08:59:39--  https://huggingface.co/facebook/sam2-hiera-large/resolve/main/sam2_hiera_large.pt
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.118, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/00/96/009696ba95ace6b12df8a9144a93766df1569c3cd7a1c886bfa431db012e2ea1/7442e4e9b732a508f80e141e7c2913437a3610ee0c77381a66658c3a445df87b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sam2_hiera_large.pt%3B+filename%3D%22sam2_hiera_large.pt%22%3B&Expires=1735203579&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNTIwMzU3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzAwLzk2LzAwOTY5NmJhOTVhY2U2YjEyZGY4YTkxNDRhOTM3NjZkZjE1NjljM2NkN2ExYzg4NmJmYTQzMWRiMDEyZTJlYTEvNzQ0MmU0ZTliNzMyYTUwOGY4MGUxNDFlN2MyOTEzNDM3YTM2MTBlZTBjNzczODFhNjY2NThjM2E0NDVkZjg3Yj9

In [1]:
import os
import torch
import numpy as np
import cv2
from sam2.build_sam import build_sam2_video_predictor
import shutil
import subprocess
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_body_keypoints(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_tensor = F.to_tensor(frame_rgb).unsqueeze(0).to('cuda')
    with torch.no_grad():
        prediction = body_detector(img_tensor)[0]
    if len(prediction['boxes']) > 0:
        best_box = prediction['boxes'][prediction['scores'].argmax()].cpu().numpy()
        x1, y1, x2, y2 = best_box
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        width, height = x2 - x1, y2 - y1
        offset_x, offset_y = width * 0.2, height * 0.2
        keypoints = np.array([
            [center_x, center_y],
            [center_x - offset_x, center_y],
            [center_x + offset_x, center_y],
            [center_x, center_y - offset_y],
            [center_x, center_y + offset_y],
        ], dtype=np.float32)
        keypoints[:, 0] = np.clip(keypoints[:, 0], x1, x2)
        keypoints[:, 1] = np.clip(keypoints[:, 1], y1, y2)
        return keypoints
    else:
        height, width = frame.shape[:2]
        center = np.array([[width // 2, height // 2]], dtype=np.float32)
        return np.tile(center, (5, 1))

def remove_background(frame, mask, bg_color):
    mask = mask.squeeze()
    if mask.dtype == bool:
        mask = mask.astype(np.uint8) * 255
    else:
        mask = (mask > 0).astype(np.uint8) * 255
    mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    bg = np.full(frame.shape, bg_color, dtype=np.uint8)
    fg = cv2.bitwise_and(frame, frame, mask=mask)
    bg = cv2.bitwise_and(bg, bg, mask=cv2.bitwise_not(mask))
    result = cv2.add(fg, bg)
    result = clean_hair_area(frame, result, mask, bg_color)
    return result

def clean_hair_area(original, processed, mask, bg_color):
    kernel = np.ones((5, 5), np.uint8)
    dilated_mask = cv2.dilate(mask, kernel, iterations=2)
    hair_edge_mask = cv2.subtract(dilated_mask, mask)
    bg_sample = cv2.bitwise_and(original, original, mask=cv2.bitwise_not(dilated_mask))
    bg_average = cv2.mean(bg_sample)[:3]
    color_distances = np.sqrt(np.sum((original.astype(np.float32) - bg_average) ** 2, axis=2))
    color_distances = (color_distances - color_distances.min()) / (color_distances.max() - color_distances.min())
    alpha = (1 - color_distances) * (hair_edge_mask / 255.0)
    alpha = np.clip(alpha, 0, 1)
    for c in range(3):
        processed[:, :, c] = processed[:, :, c] * (1 - alpha) + bg_color[c] * alpha
    return processed



In [2]:
checkpoint = 'sam2_hiera_large.pt'
model_cfg = 'sam2_hiera_l.yaml'
predictor = build_sam2_video_predictor(model_cfg, checkpoint)
body_detector = fasterrcnn_resnet50_fpn(pretrained=True)
body_detector.eval()
body_detector.to("cuda")
bg_color='#ECF0F1 '


input_video = '/content/input.mp4'
bg_color = tuple(int(bg_color.lstrip('#')[i:i + 2], 16) for i in (0, 2, 4))[::-1]
frames_dir = "/content/frames"
if os.path.exists(frames_dir):
    shutil.rmtree(frames_dir)
os.makedirs(frames_dir, exist_ok=True)
ffmpeg_cmd = ["ffmpeg", "-i", str(input_video), "-q:v", "2", "-start_number", "0",f"{frames_dir}/%05d.jpg"]
result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True, check=True)
frame_names = [p for p in os.listdir(frames_dir) if p.endswith(('.jpg', '.jpeg', '.JPG', '.JPEG'))]
frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))
inference_state = predictor.init_state(video_path=frames_dir)
first_frame_path = os.path.join(frames_dir, frame_names[0])
first_frame = cv2.imread(first_frame_path)
keypoints = detect_body_keypoints(first_frame)
_, out_obj_ids, out_mask_logits = predictor.add_new_points(inference_state=inference_state, frame_idx=0, obj_id=1, points=keypoints, labels=np.ones(len(keypoints), dtype=np.int32))
video_segments = {}
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
    video_segments[out_frame_idx] = {
        out_obj_id: out_mask_logits[i].cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }
output_frames_dir = '/content/output_frames'
os.makedirs(output_frames_dir, exist_ok=True)
frame_count = 0
for out_frame_idx in range(len(frame_names)):
    frame_path = os.path.join(frames_dir, frame_names[out_frame_idx])
    frame = cv2.imread(frame_path)
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        frame_with_bg_removed = remove_background(frame, out_mask, bg_color)
    output_frame_path = os.path.join(output_frames_dir, f"{out_frame_idx:05d}.jpg")
    cv2.imwrite(output_frame_path, frame_with_bg_removed)
    frame_count += 1
output_video_path = '/content/output.mp4'
final_video_cmd = [
    "ffmpeg", "-y",
    "-framerate", "30",
    "-i", f"{output_frames_dir}/%05d.jpg",
    "-c:v", "libx264",
    "-pix_fmt", "yuv420p",
    output_video_path
]
result = subprocess.run(final_video_cmd, capture_output=True, text=True, check=True)
print(output_video_path)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 122MB/s]
propagate in video: 100%|██████████| 91/91 [01:20<00:00,  1.13it/s]


/content/output.mp4


In [4]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)